[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zhimingkuang/Harvard-AM-115/blob/main/04_population_single_3/animate_solution.ipynb)

This notebook provide an example to create an animation of solution to a 1D diffusion equation.

Reference: https://github.com/chr1shr/am205_g_activities/blob/master/eno_methods/lin_adv_eno.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

### TL;DR: For creating the animation, skip to the last two cells
>You only need to care how the snapshot array is created and stored in the following example (look at `rho_s`). Once you have the snapshot array stored, you can run `animate_solution` to visualize the evolution of solution.


### Discretize 1D diffusion equation

We want to model how higher density diffuses with the 1D diffusion equation. The PDE is

\begin{equation}
    \frac{\partial \rho}{\partial t}=\nu\frac{\partial^2 \rho}{\partial x^2}
\end{equation}

We use central difference to discretize this equation (superscript for # time steps and subscript for spatial index), and we have

\begin{equation}
\frac{\rho^{n+1}_i-\rho^n_i}{\Delta t} = \nu \frac{\rho^n_{i+1}-2\rho^n_i+\rho^n_{i-1}}{\Delta x^2}
\end{equation}

Reorganize the terms, we have the discretized update rule to step forward in time of the density field:

\begin{equation}\boxed{
\rho^{n+1}_i = \rho^n_i + \frac{\nu\Delta t}{\Delta x^2}\left(\rho^n_{i+1}-2\rho^n_i+\rho^n_{i-1}\right)
}\end{equation}

### Simulate 1D diffusion equation

We first initialize the density field to be a box function at t=0, and we increment through time to simulate the density field in the next time step until we reach the end of the number of time steps. Here we simulate 1000 time steps, and we output every 10 time steps. So in total we should have 101 snapshots (plus an additional initial density field at t=0).

Intuitively, this box function will diffuse to a smoother shape (like normal distribution).

In [2]:
# Define simulation parameters
nx = 101 # number of gridpoints
dx = 2./(nx-1) # grid spacing
dt = 0.01 # time steps
nt = 1000 # number of time steps
nu = 0.01 # kinematic viscosity
ns = 10 # number of snapshots

# Initialization solution arrays
rho = np.ones(nx) # array for current step
rho_n = np.ones(nx) # array for the next step
# Set initial condition, here choose a box function
rho[int(nx/2-10):int(nx/2+10)+1] = 2

# Create an array for storing all snapshots
rho_s = np.empty((nx,int(nt/ns+1)))
# Store the I.C.
rho_s[:,0] = rho[:nx]

# Iterate through time
for n in range(nt):
    # Central difference method
    for i in range(1, nx-1):
        rho_n[i] = rho[i] + nu*dt/dx**2 * (rho[i+1] - 2*rho[i] + rho[i-1])
    # Note: We did not implement boundary conditions,
    # but if there is B.C.s then they should be coded here
    rho = np.copy(rho_n)
    # Store snapshots
    if n%ns==0: rho_s[:,n//ns+1] = rho[:nx] # // is integer division

In [3]:
# Customize for matplotlib
# If interested in the matplotlib object hierarchy, check: https://realpython.com/python-matplotlib-guide/
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['mathtext.default'] = 'regular'
# Change font size: http://www.futurile.net/2016/02/27/matplotlib-beautiful-plots-with-style/
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 20

### Animate 1D diffusion equation

The following code cell defines a set of helper functions that allow us to visualize the diffusion as an animation.
>For other solutions animations, you only need to recreate/store `rho_s` and change plotting labels/ranges.

In [4]:
def plot_solution(x, y, ax):
    sol, = ax.plot(x, y, color='C0')
    return sol
    
def update_solution(sol, x, y):
    sol.set_data(x, y)

def animate_solution(x, z):
    # Each column of z represents a snapshot
    # Initialize plot 
    fig, ax = plt.subplots(1,1,figsize=(8,4))
    # Initial state
    ax.plot(x, z[:,0], linestyle='dashed', color='C0')
    
    sol = plot_solution(x,z[:,0],ax)
    
    ax.set_aspect('equal')
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$\rho$')

    def animate(i):
        '''Plot updates for animation'''
        fr = i
        update_solution(sol, x, z[:,i])
        ax.set_title(r'$t = %i \Delta t$' % (i*10)) # hardcode nout here
        return sol,

    ani = animation.FuncAnimation(fig, animate, frames=z.shape[1], interval=100, blit=True)
    plt.close(fig)
    
    return ani

In [5]:
x = np.linspace(0, 2, nx)
ani = animate_solution(x, rho_s)
HTML(ani.to_html5_video())

In [6]:
# Uncomment to save animation as gif
# ani.save('animate_solution.gif', writer='pillow', fps=10)